In [4]:
# Cargo los datos
import pandas as pd

df_train = pd.read_csv('data/train_timeseries_interpolated.csv')
df_val = pd.read_csv('data/val_timeseries_interpolated.csv')
df_test = pd.read_csv('data/test_timeseries_interpolated.csv')


In [6]:
df_train.columns

Index(['fips', 'date', 'PRECTOT', 'PS', 'QV2M', 'T2M', 'T2MDEW', 'T2MWET',
       'T2M_MAX', 'T2M_MIN', 'T2M_RANGE', 'TS', 'WS10M', 'WS10M_MAX',
       'WS10M_MIN', 'WS10M_RANGE', 'WS50M', 'WS50M_MAX', 'WS50M_MIN',
       'WS50M_RANGE', 'score', 'score_interpolated',
       'score_final_interpolated'],
      dtype='object')

In [7]:
X_train = df_train.drop(columns=['fips', 'score_final_interpolated'])
y_train = df_train['score_final_interpolated']
X_val = df_val.drop(columns=['fips', 'score_final_interpolated'])
y_val = df_val['score_final_interpolated']
X_test = df_test.drop(columns=['fips', 'score_final_interpolated'])
y_test = df_test['score_final_interpolated']

In [11]:
from flaml import AutoML
from sklearn.metrics import f1_score

# Asumo que ya tienes los conjuntos X_train, y_train, X_val, y_val, X_test, y_test
# Si no, puedes cargarlos aquí.

# Concatenar el conjunto de entrenamiento y validación para darle más datos a FLAML
X = pd.concat([X_train, X_val], axis=0)
y = pd.concat([y_train, y_val], axis=0)
# Crear un objeto AutoML
automl = AutoML()

# Configurar los parámetros para FLAML
settings = {
    "time_budget": 600,  # Tiempo límite en segundos (10 minutos en este caso)
    "metric": 'f1',  # Métrica a optimizar
    "task": 'classification',  # Especificamos que es una tarea de clasificación
    "log_file_name": "flaml_timeseries_classification.log",  # Archivo de log
    "eval_method": "cv",  # Usamos validación cruzada
    "n_splits": 5,  # Número de particiones para la validación cruzada
    "estimator_list": ['lgbm', 'xgboost', 'rf'],  # Algoritmos a probar
}

# Entrenar el modelo usando FLAML en el conjunto de entrenamiento y validación
automl.fit(X, y, **settings)

# Hacer predicciones en el conjunto de prueba
y_pred = automl.predict(X_test)

# Evaluar el rendimiento del modelo en los datos de prueba
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"F1 Score (weighted) en el conjunto de prueba: {f1:.4f}")

# Mostrar el mejor modelo encontrado y otros detalles
print("Mejor estimador:", automl.best_estimator)
print("Mejor configuración de hiperparámetros:", automl.best_config)
print("Mejor métrica:", automl.best_loss)
print("Duración total:", automl.time_taken)


[flaml.automl.logger: 09-07 11:42:41] {1680} INFO - task = classification
[flaml.automl.logger: 09-07 11:42:41] {1691} INFO - Evaluation method: cv
[flaml.automl.logger: 09-07 11:42:46] {1789} INFO - Minimizing error metric: 1-f1
[flaml.automl.logger: 09-07 11:42:46] {1901} INFO - List of ML learners in AutoML Run: ['lgbm', 'xgboost', 'rf']
[flaml.automl.logger: 09-07 11:42:46] {2219} INFO - iteration 0, current learner lgbm


ValueError Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted']. happened in `metric_loss_score`, set `val_loss` to `np.inf`
ValueError Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted']. happened in `metric_loss_score`, set `val_loss` to `np.inf`
ValueError Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted']. happened in `metric_loss_score`, set `val_loss` to `np.inf`
ValueError Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted']. happened in `metric_loss_score`, set `val_loss` to `np.inf`
ValueError Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted']. happened in `metric_loss_score`, set `val_loss` to `np.inf`


[flaml.automl.logger: 09-07 11:44:07] {2345} INFO - Estimated sufficient time budget=811372s. Estimated necessary time budget=811s.
[flaml.automl.logger: 09-07 11:44:07] {2392} INFO -  at 152.6s,	estimator lgbm's best error=inf,	best estimator None's best error=inf
[flaml.automl.logger: 09-07 11:44:07] {2219} INFO - iteration 1, current learner lgbm


ValueError Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted']. happened in `metric_loss_score`, set `val_loss` to `np.inf`
ValueError Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted']. happened in `metric_loss_score`, set `val_loss` to `np.inf`
ValueError Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted']. happened in `metric_loss_score`, set `val_loss` to `np.inf`
ValueError Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted']. happened in `metric_loss_score`, set `val_loss` to `np.inf`
ValueError Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted']. happened in `metric_loss_score`, set `val_loss` to `np.inf`


[flaml.automl.logger: 09-07 11:45:24] {2392} INFO -  at 229.8s,	estimator lgbm's best error=inf,	best estimator None's best error=inf
[flaml.automl.logger: 09-07 11:45:24] {2219} INFO - iteration 2, current learner xgboost


NameError: name 'xgboost_version' is not defined

In [15]:
from flaml import AutoML
from sklearn.metrics import f1_score

# Definir una función de métrica personalizada que use 'weighted'
def weighted_f1(X_val, y_val, estimator, labels, X_train, y_train, weight_val=None, weight_train=None):
    y_pred = estimator.predict(X_val)
    return f1_score(y_val, y_pred, average='weighted')  # Usamos 'weighted' para multiclase

# Crear un objeto AutoML
automl = AutoML()

# Entrenar el modelo usando FLAML en el conjunto de entrenamiento y validación
automl.fit(X_train, y_train, task="classification")

# Hacer predicciones en el conjunto de prueba
y_pred = automl.predict(X_test)

# Evaluar el rendimiento del modelo en los datos de prueba
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"F1 Score (weighted) en el conjunto de prueba: {f1:.4f}")

# Mostrar el mejor modelo encontrado y otros detalles
print("Mejor estimador:", automl.best_estimator)
print("Mejor configuración de hiperparámetros:", automl.best_config)
print("Mejor métrica:", automl.best_loss)
print("Duración total:", automl.time_taken)

MemoryError: Unable to allocate 866. MiB for an array with shape (5678316, 20) and data type float64